# Bem-vindos
Este notebook apresenta as etapas para a extração de dados utilizados em análises estatísticas (descritivas e inferenciais) para descrição Sociolinguística. Nele, são utilizados códigos para a extração de quatro fenômenos morfossintáticos variáveis do português brasileiro, a saber:

a) Uso variável de artigo definido antes de possessivos pré-nominais;
b) Variação nos pronomes pessoais de segunda pessoa do singular (2PS);
c) Variação nos pronomes clíticos de 2PS; e
d) Variação nos pronomes possessivos de 2PS.

A extração é feito a partir de códigos provenientes spaCy 3.6 (HONIIBAL et al., 2020), biblioteca aberta de Processamento de Linguagem Natural (PLN), por meio da função Matcher (busca de tokens), que localiza o termo buscado, a partir de arquivos .txt gerados por meio de entrevistas sociolinguísticas transcritas, considerando os critérios de seleção elencados pelo pesquisador. Nesse sentido, os códigos de busca foram escritos considerando os seguintes critérios, que seguem a ordem disposta acima:

f)	Para o fenômeno em (a), apenas quando os possessivos precederem nomes;
g)	Para o fenômeno em (b), apenas quando as variantes estiverem em função de sujeito, foneticamente explícitas, precedendo ou sucedendo o verbo com o qual estabelece concordância.
h)	Para o fenômeno em (c), quando o clítico estiver em função dativa ou acusativa, em ênclise ou próclise; e
i)	Para o fenômeno em (d), quando os possessivos precederem e sucederem nomes.


Por fim, por ser uma biblioteca em linguagem Python, após a extração dos dados pelo spaCy 3.6, usamos a função data.frame da biblioteca pandas para armazenar os dados em planilha para utilização em outras plataformas, como o R.

# 1. Montar o Google Drive
Neste momento, precisamos ligar o Drive ao Notebook, de modo com que possamos acessar os arquivos do Drive aqui. Ao executar a função, uma janela será aberta para que você acesse a conta Google na qual você está usando o Google Colab, que é a mesma a qual você utilizará o Drive.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. Instalar as bibliotecas
Neste momento, são instaladas e importadas todas as bibliotecas e componentes necessários para funcionamento dos códigos.

In [4]:
!pip install -U spacy

!pip install -U spacy-lookups-data

!python -m spacy download pt_core_news_lg

!pip install pandas

!pip install wasabi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 3.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import spacy
nlp = spacy.load('pt_core_news_lg')

# 3. Carregar os arquivos .txt do Google Drive
Para tanto, é necessário:
- Criar uma pasta no Google Drive contendo os arquivos .txt que você deseja processar.
- Determinar o caminho para a pasta no Google Drive. Por exemplo, se a pasta estiver localizada em "Meu Drive/Minha Pasta", o caminho será "/content/drive/MyDrive/Minha Pasta".
- Execute o seguinte código para carregar os arquivos .txt.

Aqui é importante que todos os arquivos possuam um padrão similar de identificação. Caso contrário, podem ocorrer erros na extração de dados.

In [6]:
nomes_arq = !ls '/content/drive/MyDrive/Entrevistas/'

# 4. Extração de informação dos arquivos
O seguinte código é utilizado para a extração de informações presentes na própria entrevista, como nome do arquivo. Em nosso caso, o nome do arquivo possui todas as informações que desejamos, como informações sobre o falante (gênero, idade, deslocamento e tempo no curso) e sobre a amostra (São Cristóvão 2019 e São Cristóvão 2020).

Código para extrair informações dos nomes das entrevistas.

In [7]:
def extrai_num_ent(nome_arquivo):
    num_ent = nome_arquivo[0:65]
    return num_ent

# 5. Extração dos fenômenos
A partir daqui, lidamos com a extração das ocorrências dos fenômenos, cada um dos fenômenos isoladamente. Para tanto, esta seção apresenta:
- Código de busca e classificação
- Armazenamento dos resultados em arquivo .xlsx

## 5.1 Determinantes possessivos antecedendo nomes

In [8]:
import spacy
from spacy.matcher import Matcher
from wasabi import Printer
import pandas as pd

colunas = ['Numero_ent', 'Contexto anterior', 'Ocorrencia','Contexto Seguinte', 'Contexto']
linhas = []

nlp = spacy.load('pt_core_news_lg')

nomes_arq = !ls '/content/drive/MyDrive/Entrevistas'

for nome in nomes_arq:
  nome_refatorado = nome.replace("'", "").strip()
  arq =f'/content/drive/MyDrive/Entrevistas/{nome_refatorado}'
  corpus = open(arq).read()
  doc = nlp(corpus)
  numero_ent = extrai_num_ent(nome)

  matcher = Matcher(vocab=nlp.vocab)
  detso = [{'LEMMA': {'NOT_IN': ['meu','teu', 'seu', 'nosso']}},
           {'POS': 'DET', 'MORPH':{'IS_SUPERSET': ['PronType=Prs']}},
           {'POS': 'NOUN'}]

  matcher.add('detso', [detso])
  detrex = matcher(doc, as_spans=True)
  matches = matcher(doc)

  match = Printer()
  for match_id, start, end in matches:
    ocorrencia = doc[start:end]
    contexto = doc[start-20:end+20]
    contexto_parte1 = doc[start-20:start]
    contexto_parte2 = doc[end: end+20]
    linhas.append ([numero_ent, contexto_parte1, ocorrencia,contexto_parte2, contexto])

dataframe = pd.DataFrame(linhas, columns = colunas)
dataframe.to_excel("det.xlsx")

## 5.2 Pronomes pessoais de 2PS

In [ ]:
import spacy
from spacy.matcher import Matcher
from wasabi import Printer
import pandas as pd

colunas = ['Numero_ent', 'Contexto anterior', 'Ocorrencia','Contexto Seguinte', 'Contexto']
linhas = []

nlp = spacy.load('pt_core_news_lg')

nomes_arq = !ls '/content/drive/MyDrive/Entrevistas'

for nome in nomes_arq:
  nome_refatorado = nome.replace("'", "").strip()
  arq =f'/content/drive/MyDrive/Entrevistas/{nome_refatorado}'
  corpus = open(arq).read()
  doc = nlp(corpus)
  numero_ent = extrai_num_ent(nome)
  matcher = Matcher(vocab=nlp.vocab)
  detso = [{'ORTH': {'IN': ['você', 'cê', 'tu']}}, {'POS': 'VERB', 'MORPH': {'IS_SUPERSET': ['VerbForm=Fin']}}]
  inte = [{'ORTH': {'IN': ['você', 'cê', 'tu']}}, {}, {'POS': 'VERB', 'MORPH': {'IS_SUPERSET': ['VerbForm=Fin']}}] # com material interveniente

  matcher.add('detso', [detso])
  matcher.add('inte', [inte])
  detrex = matcher(doc, as_spans=True)
  matches = matcher(doc)
  match = Printer()
  for match_id, start, end in matches:
        ocorrencia = doc[start:end]
        contexto = doc[start-20:end+20]
        contexto_parte1 = doc[start-20:start]
        contexto_parte2 = doc[end: end+20]
        linhas.append ([numero_ent, contexto_parte1, ocorrencia,contexto_parte2, contexto])

dataframe = pd.DataFrame(linhas, columns = colunas)
dataframe.to_excel("pro.xlsx")

## 5.3 Clíticos de 2PS

In [ ]:
import spacy
from spacy.matcher import Matcher
from wasabi import Printer
import pandas as pd

colunas = ['Numero_ent', 'Contexto anterior', 'Ocorrencia','Contexto Seguinte', 'Contexto']
linhas = []

nlp = spacy.load('pt_core_news_lg')

nomes_arq = !ls '/content/drive/MyDrive/Entrevistas'

for nome in nomes_arq:
  nome_refatorado = nome.replace("'", "").strip()
  arq =f'/content/drive/MyDrive/Entrevistas/{nome_refatorado}'
  corpus = open(arq).read()
  doc = nlp(corpus)
  numero_ent = extrai_num_ent(nome)
  matcher = Matcher(vocab=nlp.vocab)
  matcher = Matcher(vocab=nlp.vocab)
  detso = [{'ORTH': {'IN': ['te', 'lhe']}}, {'POS': 'VERB'}]
  clitc= [{'POS': 'VERB'}, {'ORTH': {'IN': ['te', 'lhe']}}]
  encli= [{'POS': 'VERB'}, {'IS_PUNCT': True}, {'ORTH': {'IN': ['te', 'lhe']}}]

  matcher.add('detso', [detso])
  matcher.add('clitc', [clitc])
  matcher.add('encli', [encli])
  detrex = matcher(doc, as_spans=True)
  matches = matcher(doc)
  match = Printer()
  for match_id, start, end in matches:
        ocorrencia = doc[start:end]
        contexto = doc[start-20:end+20]
        contexto_parte1 = doc[start-20:start]
        contexto_parte2 = doc[end: end+20]
        linhas.append ([numero_ent, contexto_parte1, ocorrencia,contexto_parte2, contexto])

dataframe = pd.DataFrame(linhas, columns = colunas)
dataframe.to_excel("cli.xlsx")

## 5.4 Possessivos de 2PS

In [ ]:
import spacy
from spacy.matcher import Matcher
from wasabi import Printer
import pandas as pd

colunas = ['Numero_ent', 'Contexto anterior', 'Ocorrencia','Contexto Seguinte', 'Contexto']
linhas = []

nlp = spacy.load('pt_core_news_lg')

nomes_arq = !ls '/content/drive/MyDrive/Entrevistas'

for nome in nomes_arq:
  nome_refatorado = nome.replace("'", "").strip()
  arq =f'/content/drive/MyDrive/Entrevistas/{nome_refatorado}'
  corpus = open(arq).read()
  doc = nlp(corpus)
  numero_ent = extrai_num_ent(nome)

  matcher = Matcher(vocab=nlp.vocab)
  pre = [{"TEXT": {"IN":["seu", "sua", "seus", "suas", "teu", "tua", "teus", "tuas"]}}, {"POS": "NOUN"}]
  pos = [{"POS": "NOUN"}, {"TEXT": {"IN":["seu", "sua", "seus", "suas", "teu", "tua", "teus", "tuas"]}}]

  matcher.add('pre', [pre])
  matcher.add('pos', [pos])
  detrex = matcher(doc, as_spans=True)
  matches = matcher(doc)
  match = Printer()
  for match_id, start, end in matches:
    ocorrencia = doc[start:end]
    contexto = doc[start-20:end+20]
    contexto_parte1 = doc[start-20:start]
    contexto_parte2 = doc[end: end+20]
    linhas.append ([numero_ent, contexto_parte1, ocorrencia,contexto_parte2, contexto])

dataframe = pd.DataFrame(linhas, columns = colunas)
dataframe.to_excel("pos.xlsx")